In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install torch

In [ ]:
pip install transformers

In [ ]:
pip install openai

In [ ]:
pip install datasets

In [ ]:
pip install peft

In [ ]:
pip install transformers huggingface-hub

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
import torch



In [ ]:
#Importing Model 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer 

#loading gemma 2B model and tokenizer 
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map = "auto", torch_dtype = "auto")



In [ ]:
import torch
model_name = "google/gemma-2b"

# Load the model and tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu", torch_dtype="auto")

# Sample question
question = "What are the symptoms of pneumonia?"

# Tokenize input
inputs = tokenizer(question, return_tensors="pt").to("cpu")

# Inference
with torch.no_grad():
    output = model.generate(**inputs, max_length=300, do_sample = True, temperature=0.7, top_p=0.9)

# Decode and print the output
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print("Answer:", answer)

In [ ]:
#Loading datasets

In [ ]:
import datasets

In [ ]:
from datasets import load_dataset

datasetTrain = load_dataset("medmcqa", split = "train")

print(datasetTrain)

In [ ]:
datasetTest = load_dataset("medmcqa", split = "test")

print(datasetTest)


In [ ]:
datasetValidate = load_dataset("medmcqa", split = "validation")

print(datasetValidate)

In [ ]:
#newDataSet = load_dataset("json", data_files = {"intents.json"})

#dataset_split = newDataSet.train_test_split(test_size = 0.2)

#train_dataset = dataset_split["train"]

#val_dataset = dataset_split["test"]


#print(train_dataset)
#print(val_dataset)

In [ ]:
# tokenizing datasets

In [ ]:
#tokenize function

In [ ]:
def preprocess_function(examples):
    inputs = [
        f"Question: {q} Options: A. {a} B. {b} C. {c} D. {d} Explanation: {exp} Answer: {chr(64 + int(cop))}"
        for q, a, b, c, d, exp, cop in zip(
            examples["question"], 
            examples["opa"], 
            examples["opb"], 
            examples["opc"], 
            examples["opd"],
            examples["exp"],
            examples["cop"]
        )
    ]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

In [ ]:
tokenized_dataset_train = datasetTrain.map(preprocess_function, batched=True)

In [ ]:
# Tokenize and prepare the dataset
tokenized_dataset_validate = datasetValidate.map(preprocess_function, batched=True)


In [ ]:
# Tokenize and prepare the dataset
tokenized_dataset_test = datasetTest.map(preprocess_function, batched=True)

print(tokenized_dataset_test)

In [ ]:
train_dataset = tokenized_dataset_train
val_dataset = tokenized_dataset_validate



In [ ]:
# Ensure that all datasets are tokenized properly and have the labels
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import logging

training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    disable_tqdm=False,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,  # your loaded/initialized model
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer  # if you're using one
)

In [ ]:
print("Starting trainer...")
trainer.train()
print("Training complete.")